In [2]:
!pip install fastapi

     |████████████████████████████████| 51kB 3.7MB/s 
     |████████████████████████████████| 10.1MB 10.9MB/s 
     |████████████████████████████████| 61kB 6.7MB/s 


In [1]:
!pip install colabcode 

In [2]:
from pydantic import BaseModel

class Match(BaseModel):
    hometeam: str 
    awayteam: str 
    attendance: int
    class Config:
        schema_extra = {
            "example": {
                "hometeam": "Bournemouth", 
                "awayteam": "Crystal Palace", 
                "attendance": 23850
            }
        }

In [3]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

# Read football stats dataset from CSV File
dataset = pd.read_csv('/content/drive/My Drive/FullDataset.csv')

Mounted at /content/drive


In [13]:
import json
preds = {}
preds['matches'] = []

with open('preds.json', 'w') as outfile:
    json.dump(preds, outfile)

In [14]:
def insert_inputs(hometeam,awayteam,attendance):
  #Latest home team match
  hteamdf = dataset.loc[(dataset['HomeTeam'] == hometeam)]
  hteamdf = hteamdf.iloc[[-1,]]
  #Latest away team match
  ateamdf = dataset.loc[(dataset['AwayTeam'] == awayteam)]
  ateamdf = ateamdf.iloc[[-1,]]
  #Assign home team name to a var
  awayhometeam = ateamdf['HomeTeam']
  #assign attendance from input
  ateamdf['Attendance'] = attendance
  #Check if the last match of away team is with the same home team
  if hometeam == awayhometeam.values:
    print(hteamdf)
  else:
    hteamdf['ATP'] = ateamdf['ATP'].values
    hteamdf['AM1_D'] = ateamdf['AM1_D'].values
    hteamdf['AM1_L'] = ateamdf['AM1_L'].values
    hteamdf['AM1_W'] = ateamdf['AM1_W'].values
    hteamdf['AM2_D'] = ateamdf['AM2_D'].values
    hteamdf['AM2_L'] = ateamdf['AM2_L'].values
    hteamdf['AM2_W'] = ateamdf['AM2_W'].values
    hteamdf['AM3_D'] = ateamdf['AM3_D'].values
    hteamdf['AM3_L'] = ateamdf['AM3_L'].values
    hteamdf['AM3_W'] = ateamdf['AM3_W'].values
    return hteamdf

In [7]:
from fastapi import FastAPI
import pickle
from fastapi.middleware.cors import CORSMiddleware
import json


app = FastAPI()

origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.on_event("startup")
def load_model():
    global model
    model = pickle.load(open("fypmodelrf", "rb"))

@app.get('/')
def index():
    return {'message': 'Home Page of the API'}
@app.get('/pastpredictions')
def getPastPred():
  with open('preds.json') as json_file:
    preds = json.load(json_file)
  return preds

@app.post('/predict')
def getPrediction(data: Match):
    result = data.dict()
    hometeam = result['hometeam']
    awayteam = result['awayteam']
    attendance = result['attendance']
    inputsdf = insert_inputs(hometeam,awayteam,attendance)
    if set(['Unnamed: 0']).issubset(inputsdf.columns):
      inputsdf = inputsdf.drop(columns=['Unnamed: 0'])
    if set(['HomeTeam', 'AwayTeam']).issubset(inputsdf.columns):
      inputsdf = inputsdf.drop(columns=['HomeTeam','AwayTeam'])
    pred_name = model.predict(inputsdf)
    pred_value = pred_name[0]
    preds['matches'].append({
      'hometeam': hometeam,
      'awayteam': awayteam,
      'prediction': pred_value 
    })

    with open('preds.json', 'w') as outfile:
      json.dump(preds, outfile)
    return {'prediction': pred_value}

In [8]:
from colabcode import ColabCode
server = ColabCode(port=10000, code=False)

In [11]:
server.run_app(app=app)

Public URL: NgrokTunnel: "http://70fc3248f2ab.ngrok.io" -> "http://localhost:10000"


INFO:     Started server process [134]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:10000 (Press CTRL+C to quit)


INFO:     203.189.185.168:0 - "GET /pastpredictions HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/uvicorn/protocols/http/h11_impl.py", line 394, in run_asgi
    result = await app(self.scope, self.receive, self.send)
  File "/usr/local/lib/python3.7/dist-packages/uvicorn/middleware/proxy_headers.py", line 45, in __call__
    return await self.app(scope, receive, send)
  File "/usr/local/lib/python3.7/dist-packages/fastapi/applications.py", line 199, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.7/dist-packages/starlette/applications.py", line 111, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.7/dist-packages/starlette/middleware/errors.py", line 181, in __call__
    raise exc from None
  File "/usr/local/lib/python3.7/dist-packages/starlette/middleware/errors.py", line 159, in __call__
    await self.app(scope, receive, _send)
  File "/usr/local/lib/python3.7/dist-pa

INFO:     203.189.185.168:0 - "GET /pastpredictions HTTP/1.1" 500 Internal Server Error


...

ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/uvicorn/protocols/http/h11_impl.py", line 394, in run_asgi
    result = await app(self.scope, self.receive, self.send)
  File "/usr/local/lib/python3.7/dist-packages/uvicorn/middleware/proxy_headers.py", line 45, in __call__
    return await self.app(scope, receive, send)
  File "/usr/local/lib/python3.7/dist-packages/fastapi/applications.py", line 199, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.7/dist-packages/starlette/applications.py", line 111, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.7/dist-packages/starlette/middleware/errors.py", line 181, in __call__
    raise exc from None
  File "/usr/local/lib/python3.7/dist-packages/starlette/middleware/errors.py", line 159, in __call__
    await self.app(scope, receive, _send)
  File "/usr/local/lib/python3.7/dist-pa

INFO:     203.189.185.168:0 - "POST /predict HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/uvicorn/protocols/http/h11_impl.py", line 394, in run_asgi
    result = await app(self.scope, self.receive, self.send)
  File "/usr/local/lib/python3.7/dist-packages/uvicorn/middleware/proxy_headers.py", line 45, in __call__
    return await self.app(scope, receive, send)
  File "/usr/local/lib/python3.7/dist-packages/fastapi/applications.py", line 199, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.7/dist-packages/starlette/applications.py", line 111, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.7/dist-packages/starlette/middleware/errors.py", line 181, in __call__
    raise exc from None
  File "/usr/local/lib/python3.7/dist-packages/starlette/middleware/errors.py", line 159, in __call__
    await self.app(scope, receive, _send)
  File "/usr/local/lib/python3.7/dist-pa

INFO:     203.189.185.168:0 - "POST /predict HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/uvicorn/protocols/http/h11_impl.py", line 394, in run_asgi
    result = await app(self.scope, self.receive, self.send)
  File "/usr/local/lib/python3.7/dist-packages/uvicorn/middleware/proxy_headers.py", line 45, in __call__
    return await self.app(scope, receive, send)
  File "/usr/local/lib/python3.7/dist-packages/fastapi/applications.py", line 199, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.7/dist-packages/starlette/applications.py", line 111, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.7/dist-packages/starlette/middleware/errors.py", line 181, in __call__
    raise exc from None
  File "/usr/local/lib/python3.7/dist-packages/starlette/middleware/errors.py", line 159, in __call__
    await self.app(scope, receive, _send)
  File "/usr/local/lib/python3.7/dist-pa